In [53]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

In [69]:
df= pd.read_csv("data prep/datasets/2024_pregame.csv")

# X = df.drop("EI", axis=1)
# X=df.drop(["FGM_x","FGA_x","FG_PCT_x","FG3A_x","FG3_PCT_x","FTM_x","FTA_x","FT_PCT_x","OREB_x","DREB_x","FGM_y","FGA_y","FG_PCT_y","FG3A_y","FG3_PCT_y","FTM_y","FTA_y","FT_PCT_y","OREB_y","DREB_y","BLKA_x","PFD_x","BLKA_y","PFD_y","EI"], axis=1)

# X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","REB_x","AST_x","TOV_x","STL_x","BLK_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","REB_y","AST_y","TOV_y","STL_y","BLK_y","PTS_y","PLUS_MINUS_y"]]

# X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","REB_x","AST_x","TOV_x","STL_x","BLK_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","REB_y","AST_y","TOV_y","STL_y","BLK_y","PTS_y","PLUS_MINUS_y"]]

X= df[["TIME","RIVALRY","NATIONAL_TV","DAY_Friday","DAY_Monday","DAY_Saturday","DAY_Sunday","DAY_Thursday","DAY_Tuesday","DAY_Wednesday","LEADERS_x","W_PCT_x","PTS_x","PLUS_MINUS_x","LEADERS_y","W_PCT_y","PTS_y","PLUS_MINUS_y"]]

# X= X.iloc[:,6:]
y = df["EI"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
X.shape

(1229, 18)

In [52]:
model = LinearRegression()
model.fit(X_train, y_train)

# model = RandomForestRegressor()
# model.fit(X_train, y_train)

# model = XGBRegressor()
# model.fit(X_train, y_train)

y_pred = model.predict(X_test)


print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))


MAE: 0.12619903189323883
RMSE: 0.14711461641131343
R²: -0.005550599029890968


In [43]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Ridge & Lasso Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred = ridge.predict(X_test_scaled)

# lasso = Lasso(alpha=0.1)
# lasso.fit(X_train_scaled, y_train)
# y_pred = lasso.predict(X_test_scaled)

# svr = SVR(kernel="rbf", C=1.0, epsilon=0.1)
# svr.fit(X_train_scaled, y_train)
# y_pred = svr.predict(X_test_scaled)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))

MAE: 0.12681835807606517
RMSE: 0.1456833853842678
R²: 0.013919588419266105


In [70]:
#DNN

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(18, 128)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(128, 256)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(256, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

# converting numpy's 64-bit floats to torch's 32-bits floats
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32).reshape(-1, 1)

model = DNN()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training it
n_epochs =260
batch_size = 16

for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i + batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i + batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 20 == 0 or epoch == (n_epochs - 1):
        print(f'Finished epoch {epoch}, latest loss {loss}')

model.eval()
with torch.no_grad():
    y_pred = model(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))


Finished epoch 0, latest loss 0.4939560890197754
Finished epoch 20, latest loss 0.48265284299850464
Finished epoch 40, latest loss 0.4865579307079315
Finished epoch 60, latest loss 0.49061769247055054
Finished epoch 80, latest loss 0.49233278632164
Finished epoch 100, latest loss 0.4982706606388092
Finished epoch 120, latest loss 0.4871646463871002
Finished epoch 140, latest loss 0.4839947819709778
Finished epoch 160, latest loss 0.47952327132225037
Finished epoch 180, latest loss 0.47014865279197693
Finished epoch 200, latest loss 0.46453025937080383
Finished epoch 220, latest loss 0.4565243422985077
Finished epoch 240, latest loss 0.44892460107803345
Finished epoch 259, latest loss 0.4467806816101074
MAE: 0.13397917
RMSE: 0.16418393
R²: -0.2524304125266792
